In [26]:
import pandas as pd
import numpy as np
import os

In [2]:
data = pd.read_csv('../data/example_data.csv')

In [13]:
data.columns = ['template', 'caption_1', 'caption_2']

In [14]:
data.head()

,template,caption_1,caption_2
0,success_kid,Didnt study for a test,still get a higher grade than someone who did
1,success_kid,Ate spaghetti with a white shirt on,no stains
2,success_kid,New neighbors,Free Wif
3,awkward_seal,You laugh when your friend says something,He was being serious
4,awkward_seal,took a photo,camera the wrong way


In [16]:
def gen_process_column(row):
    return row.caption_1 + '|' + row.caption_2 + '|'
    

In [17]:
data['txt'] = data.apply(gen_process_column, axis = 1)

In [27]:
def save_npy(folder, file_name, columns, df):
    """
    Permite crear los archivos de entrenamiento, prueba y validación. 
    Es necesario definir las columnas que van a ser utilizadas. 
    
    Params
    ---------
        folder (str):
            Ruta donde se van a guardar los archivos.
        
        file_name (str):
            Nombre del archivo de destino.
            
        columns (list):
            Una lista de columnas que contiene la información a exportar
            
        df (pandas.DataFrame):
            Un DataFrame que contenga la información para procesar
    """
    save_path = os.path.join(folder, file_name)
    array = df[columns].values
    np.save(save_path, array)

In [28]:
save_npy('../data', 'prueba.npy', columns = ['template', 'txt'], df = data)

In [32]:
train_data = np.load(os.path.join('../data', 'prueba.npy'), allow_pickle = True)

In [92]:

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
label_id_counter = 0
for i, row in enumerate(train_data):
    template_id = str(row[0]).zfill(12)
    text = row[1].lower()
    start_index = len(template_id) + 2 + 1 + 2  # template_id, spaces, box_index, spaces
    box_index = 0
    for j in range(0, len(text)):
        char = text[j]
        # note: it is critical that the number of spaces plus len(box_index) is >= the convolution width
        texts.append(template_id + '  ' + str(box_index) + '  ' + text[0:j])
        if char in labels_index:
            label_id = labels_index[char]
        else:
            label_id = label_id_counter
            labels_index[char] = label_id
            label_id_counter += 1
        labels.append(label_id)
        if char == '|':
            box_index += 1

    if i >= 4:
        break

In [42]:
def map_char_to_int(texts):
    char_counts = {}
    for text in texts:
        for char in text:
            char_counts[char] = char_counts[char] + 1 if char in char_counts else 1
    char_counts_sorted = sorted(char_counts.items(), key=operator.itemgetter(1), reverse=True)
    char_to_int = {}
    for i, row in enumerate(char_counts_sorted):
        char_to_int[row[0]] = i + 1
    return char_to_int

In [47]:
vocab = map_char_to_int(texts)

In [44]:
import operator

In [48]:
def texts_to_sequences(texts, char_to_int):
    sequences = []
    for text in texts:
        sequences.append([char_to_int[char] for char in text])
    return sequences

In [50]:
seqs = texts_to_sequences(texts, vocab)

In [66]:
labels = [char_to_int[char] for char in labels]

NameError: name 'char_to_int' is not defined

In [63]:
labels_index 

{' ': 0,
 'd': 1,
 'i': 2,
 'n': 3,
 't': 4,
 's': 5,
 'u': 6,
 'y': 7,
 'f': 8,
 'o': 9,
 'r': 10,
 'a': 11,
 'e': 12,
 '|': 13,
 'l': 14,
 'g': 15,
 'h': 16,
 'm': 17,
 'w': 18,
 'p': 19,
 'b': 20,
 'k': 21,
 'c': 22,
 '’': 23}

In [67]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
 tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

# Codificadores de Texto

In [ ]:
import tensorflow as tf
import numpy as np

In [68]:
train_data = np.load(os.path.join('../data', 'prueba.npy'), allow_pickle = True)

In [86]:
vocab = sorted(set("".join(data.txt.values.tolist()).lower()))

In [88]:
tf.strings.unicode_split(texts, input_encoding='UTF-8')

NameError: name 'tf' is not defined

In [95]:
texts

['0success_kid  0  ',
 '0success_kid  0   ',
 '0success_kid  0   d',
 '0success_kid  0   di',
 '0success_kid  0   did',
 '0success_kid  0   didn',
 '0success_kid  0   didnt',
 '0success_kid  0   didnt ',
 '0success_kid  0   didnt s',
 '0success_kid  0   didnt st',
 '0success_kid  0   didnt stu',
 '0success_kid  0   didnt stud',
 '0success_kid  0   didnt study',
 '0success_kid  0   didnt study ',
 '0success_kid  0   didnt study f',
 '0success_kid  0   didnt study fo',
 '0success_kid  0   didnt study for',
 '0success_kid  0   didnt study for ',
 '0success_kid  0   didnt study for a',
 '0success_kid  0   didnt study for a ',
 '0success_kid  0   didnt study for a t',
 '0success_kid  0   didnt study for a te',
 '0success_kid  0   didnt study for a tes',
 '0success_kid  0   didnt study for a test',
 '0success_kid  1   didnt study for a test|',
 '0success_kid  1   didnt study for a test| ',
 '0success_kid  1   didnt study for a test| s',
 '0success_kid  1   didnt study for a test| st',
 '0suc